In [1]:
"""
This is the script we use to generate our schedulers clean accuracy, adversarial accuracy, runtime and others.
we averaged multiple runs in our repport
"""

'\nThis is the script we use to generate our schedulers clean accuracy, adversarial accuracy, runtime and others.\nwe averaged multiple runs in our repport\n'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os
#sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '../../..')))

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import model.Models as Models
import Defences as Defences
import schedulers.Schedulers as Schedulers
import pandas as pd
import numpy as np
from Attacks import pgd_attack
import torch.nn.functional as F

In [4]:
"""
Hyper parameters, feel free to experiment.
The commented ones are the ones we used on CIFAR10
"""

device = "cuda"

# MNIST SetUp ------------------------

dataset = "MNIST" 
batchsize = 64 

k_max = 20 
epsilon = 0.3 
epochs = 8 
lr = 1e-3

def get_model():
    return Models.SmallConvNet().to(device) # See our Models.file, we also implemented ResNets and such

def get_optimizer(model, lr = lr):
    return torch.optim.Adam(model.parameters(), lr=lr)

# CIFAR SetUp ------------------------

"""
dataset = "CIFAR10"
batchsize = 200

k_max = 7
epsilon = 255/8
epochs = 20
lr = 1e-3

def get_model():
    return Models.resnet18_cifar10().to(device) # See our Models.file, we also implemented ResNets and such


def get_optimizer(model, lr = lr):
    return torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
"""

'\ndataset = "CIFAR10"\nbatchsize = 200\n\nk_max = 7\nepsilon = 255/8\nepochs = 20\nlr = 1e-3\n\ndef get_model():\n    return Models.resnet18_cifar10().to(device) # See our Models.file, we also implemented ResNets and such\n\n\ndef get_optimizer(model, lr = lr):\n    return torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)\n'

In [5]:
def get_data_loaders(batch_size=64, dataset = dataset):
    if dataset == "MNIST":
        print("Loading MNIST data...")
        transform = transforms.ToTensor()
        train_dataset = datasets.MNIST(root="../../project_code/data", train=True, download=True, transform=transform)
        test_dataset = datasets.MNIST(root="../../project_code/data", train=False, download=True, transform=transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)
        print("MNIST data loaded.")
        return train_loader, test_loader

    elif dataset == "CIFAR10":
        """Get CIFAR-10 data loaders with appropriate transforms"""
        # CIFAR-10 specific transforms
        transform_train = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ])
        
        transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ])
        
        train_dataset = datasets.CIFAR10(root="../data", train=True, download=True, transform=transform_train)
        test_dataset = datasets.CIFAR10(root="../data", train=False, download=True, transform=transform_test)
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
        test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False, num_workers=2)
        
        return train_loader, test_loader
    

In [6]:
def evaluate_model(model, test_loader, criterion, device):
    """
    Evaluate model on clean examples
    """
    model.eval()
    test_loss = 0.0
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
    test_loss /= test_total
    test_acc = 100.0 * test_correct / test_total
    return test_loss, test_acc

In [7]:
def run_all_k_strategies(k_min=0, k_max=k_max, epsilon=epsilon, num_epochs=epochs, device=device, lr = lr, dataset = dataset):
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    print(f"Training on {device}")
    
    train_loader, test_loader = get_data_loaders(dataset = dataset)
    results = []
    schedulers = {
        "Vanilla": Schedulers.VanillaScheduler(),
        "Constant": Schedulers.ConstantScheduler(k_min, k_max),
        "Linear": Schedulers.LinearScheduler(k_min, k_max),
        "LinearUniformMix": Schedulers.LinearUniformMixScheduler(k_min, k_max),
        "Cyclic": Schedulers.CyclicScheduler(k_min, k_max),
        "Exponential": Schedulers.ExponentialScheduler(k_min, k_max),
        "Random": Schedulers.RandomScheduler(k_min, k_max),
        "Constant": Schedulers.ConstantScheduler(k_min, k_max),
    }
    
    os.makedirs("results", exist_ok=True)
    for name, scheduler in schedulers.items():
        print(f"\nTraining with {name} scheduler...")
        
        #model = Models.MediumConvNet().to(device)  # Using larger model
        #optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        model = get_model()
        optimizer = get_optimizer(model)
        
        criterion = nn.CrossEntropyLoss()
        Defences.train_with_adversarial_scheduler(
            model, train_loader, test_loader, optimizer, criterion,
            epsilon, scheduler, device, num_epochs=num_epochs, test_eval_rate=2, 
            clip_norm = 1, sched_lr = True
        )
        test_loss, test_acc = evaluate_model(model, test_loader, criterion, device)
        results.append({
            "strategy": name,
            "test_loss": test_loss,
            "test_accuracy": test_acc
        })
        # Save model after training
        torch.save(model.state_dict(), f"results/model_{name}.pth")
    # Save results to CSV
    pd.DataFrame(results).to_csv("results/k_strategy_results.csv", index=False)
    print("\nSummary of results:")
    print(pd.DataFrame(results))

In [8]:
def load_models(strategies, device):
    model_dict = {}
    for name in strategies:
        model = get_model()  
        model.load_state_dict(torch.load(f"results/model_{name}.pth", map_location=device))
        model.eval()
        model_dict[name] = model
    return model_dict


def evaluate_strategies_on_attacks(model_dict, test_loader, device, epsilon=epsilon, k_list=[1,2,4,8,16]):
    results = []
    criterion = nn.CrossEntropyLoss()
    for strategy, model in model_dict.items():
        # Clean accuracy
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        clean_acc = 100.0 * correct / total

        # Adversarial accuracy for each k
        for k in k_list:
            correct_adv = 0
            total_adv = 0
            confidences = []
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                adv_images, _ = pgd_attack(images, labels, model, criterion, epsilon, k, device)
                outputs = model(adv_images)
                probs = F.softmax(outputs, dim=1)
                _, predicted = outputs.max(1)
                total_adv += labels.size(0)
                correct_adv += (predicted == labels).sum().item()
                # Mean confidence on correct predictions
                for i in range(labels.size(0)):
                    confidences.append(probs[i, predicted[i]].item())
            adv_acc = 100.0 * correct_adv / total_adv
            mean_conf = float(np.mean(confidences)) if confidences else 0.0
            results.append({
                "strategy": strategy,
                "k": k,
                "clean_acc": clean_acc,
                "adv_acc": adv_acc,
                "mean_confidence": mean_conf
            })
    df = pd.DataFrame(results)
    df.to_csv("results/adversarial_evaluation.csv", index=False)
    print("\nAdversarial evaluation results:")
    print(df)
    return df

In [9]:
"""
Generate results
"""
run_all_k_strategies()

Training on cuda
Loading MNIST data...
MNIST data loaded.

Training with Vanilla scheduler...
Epoch 1/8 | Train Loss: 0.3349, Train Acc: 90.12%
Epoch 2/8 | Train Loss: 0.0933, Train Acc: 97.09% | Test Loss: 0.0575, Test Acc: 98.15%
Test Adv Loss (k=7): 13.1916, Test Adv Acc: 0.00%
Current LR: 0.000750
Epoch 3/8 | Train Loss: 0.0631, Train Acc: 98.05%
Epoch 4/8 | Train Loss: 0.0507, Train Acc: 98.43% | Test Loss: 0.0434, Test Acc: 98.42%
Test Adv Loss (k=7): 16.1820, Test Adv Acc: 0.00%
Current LR: 0.000500
Epoch 5/8 | Train Loss: 0.0420, Train Acc: 98.73%
Epoch 6/8 | Train Loss: 0.0355, Train Acc: 98.93% | Test Loss: 0.0330, Test Acc: 98.86%
Test Adv Loss (k=7): 18.2966, Test Adv Acc: 0.00%
Current LR: 0.000250
Epoch 7/8 | Train Loss: 0.0301, Train Acc: 99.10%
Epoch 8/8 | Train Loss: 0.0260, Train Acc: 99.26% | Test Loss: 0.0364, Test Acc: 98.84%
Test Adv Loss (k=7): 19.8589, Test Adv Acc: 0.00%
Current LR: 0.000000
Training time (without eval): 0 min 57.86 sec

Training with Constant 

In [10]:
# Evaluation phase

_, test_loader = get_data_loaders()
strategies = ["Vanilla", "Constant", "Linear", "LinearUniformMix", "Exponential", "Cyclic", "Random"]
#strategies = ["Vanilla", "LinearUniformMix", "Cyclic"]
model_dict = load_models(strategies, device)
evaluate_strategies_on_attacks(model_dict, test_loader, device)

Loading MNIST data...
MNIST data loaded.

Adversarial evaluation results:
            strategy   k  clean_acc  adv_acc  mean_confidence
0            Vanilla   1      98.84     0.29         0.877280
1            Vanilla   2      98.84     0.01         0.956656
2            Vanilla   4      98.84     0.00         0.985394
3            Vanilla   8      98.84     0.00         0.992300
4            Vanilla  16      98.84     0.00         0.994804
5           Constant   1      51.28    41.06         0.345403
6           Constant   2      51.28    39.07         0.344707
7           Constant   4      51.28    37.98         0.344060
8           Constant   8      51.28    37.43         0.343125
9           Constant  16      51.28    37.33         0.342866
10            Linear   1      96.53    89.11         0.861458
11            Linear   2      96.53    87.36         0.849682
12            Linear   4      96.53    85.52         0.838642
13            Linear   8      96.53    84.56         0.831

,strategy,k,clean_acc,adv_acc,mean_confidence
0,Vanilla,1,98.84,0.29,0.877280
1,Vanilla,2,98.84,0.01,0.956656
2,Vanilla,4,98.84,0.00,0.985394
3,Vanilla,8,98.84,0.00,0.992300
4,Vanilla,16,98.84,0.00,0.994804
5,Constant,1,51.28,41.06,0.345403
6,Constant,2,51.28,39.07,0.344707
7,Constant,4,51.28,37.98,0.344060
8,Constant,8,51.28,37.43,0.343125
9,Constant,16,51.28,37.33,0.342866
